[IPython Notebook](compute.ipynb) |  [Python Script](compute.py)

Compute
============================

Now that we have datasets added to our Bundle, our next step is to run the forward model and compute a synthetic model for each of these datasets.

Setup
-----------------------------

As always, let's do imports and initialize a logger and a new Bundle.  See [Building a System](building_a_system.html) for more details.

In [1]:
import phoebe
from phoebe import u # units
import numpy as np
import matplotlib.pyplot as plt

logger = phoebe.logger(clevel='INFO')

b = phoebe.default_binary()

/usr/local/lib/python2.7/dist-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


/usr/local/lib/python2.7/dist-packages/astropy/units/quantity.py:732: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  return super(Quantity, self).__eq__(other)


And we'll attach some dummy datasets.  See [Datasets](datasets.html) for more details.

In [2]:
b.add_dataset(phoebe.dataset.orb, time=np.linspace(0,10,10), dataset='orb01', component=['primary', 'secondary'])

times, fluxes, errors = np.loadtxt('test.lc.in', unpack=True)

# test.lc.in has 1000 datapoints... for now we'll trim that until phoebe is optimized
# TODO: remove this line once reasonable
times, fluxes, errors = times[:10], fluxes[:10], errors[:10]

b.add_dataset(phoebe.dataset.lc, time=times, flux=fluxes, sigma=errors, dataset='lc01')

<ParameterSet: 11 parameters | kinds: LC, LC_dep>

Adding Compute Options
--------------------------------------

First we need to create the options that will be used when we compute the model.
This syntax should look very familiar by now, it takes a function (or the name of a recognized function in phoebe.parameters.compute) and then any
kwargs to set in that ParameterSet.

Let's say that we want to create two sets of compute options - in this example, we'll create one called 'preview' which will cut some corners to quickly get us a model, and one called 'detailed' which will get a much more precise model but likely take longer.  As with other tags, the string you provide for the compute tag is up to you (so long as it doesn't raise an error because it conflicts with other tags).

In [3]:
b.add_compute(phoebe.compute.phoebe, compute='preview', refl=False)

<ParameterSet: 20 parameters | datasets: lc01, orb01>

In [4]:
print b['preview@compute']

ParameterSet: 20 parameters
  dynamics_method@preview@com...: keplerian
            ltte@preview@compute: False
            refl@preview@compute: False
      store_mesh@preview@compute: False
     mesh_offset@preview@compute: True
  eclipse_method@preview@compute: native
    enabled@lc01@preview@compute: True
   enabled@orb01@preview@compute: True
  mesh_method@primary@preview...: marching
  mesh_method@secondary@previ...: marching
   delta@primary@preview@compute: 0.1
  delta@secondary@preview@com...: 0.1
  maxpoints@primary@preview@c...: 100000
  maxpoints@secondary@preview...: 100000
  distortion_method@primary@p...: roche
  distortion_method@secondary...: roche
     atm@primary@preview@compute: ck2004
   atm@secondary@preview@compute: ck2004
       lc_method@preview@compute: numerical
      fti_method@preview@compute: None


In [5]:
b.add_compute('phoebe', compute='detailed', refl=True)

<ParameterSet: 20 parameters | datasets: lc01, orb01>

In [6]:
print b.get_compute('detailed')

ParameterSet: 20 parameters
  dynamics_method@detailed@co...: keplerian
           ltte@detailed@compute: False
           refl@detailed@compute: True
     store_mesh@detailed@compute: False
    mesh_offset@detailed@compute: True
  eclipse_method@detailed@com...: native
   enabled@lc01@detailed@compute: True
  enabled@orb01@detailed@compute: True
  mesh_method@primary@detaile...: marching
  mesh_method@secondary@detai...: marching
  delta@primary@detailed@compute: 0.1
  delta@secondary@detailed@co...: 0.1
  maxpoints@primary@detailed@...: 100000
  maxpoints@secondary@detaile...: 100000
  distortion_method@primary@d...: roche
  distortion_method@secondary...: roche
    atm@primary@detailed@compute: ck2004
  atm@secondary@detailed@compute: ck2004
      lc_method@detailed@compute: numerical
     fti_method@detailed@compute: None


Editing Compute Options
-------------------------------------

### Backend-Specific Compute Options

Most of the parameters in the compute options are specific to the backend being used.  Here, of course, we're using the PHOEBE 2.0 backend - but for details on other backends see the [Alternate Backends Tutorial](alternate_backends).

The PHOEBE 2.0 compute options are described in the tutorial on their relevant dataset types:

* [Orbits (ORB)](ORB)
* [Meshes (MESH)](MESH)
* [Light Curves/Fluxes (LC)](LC)
* [Radial Velocities (RV)](RV)
* [Eclipse Timing Variations (ETV)](ETV)

### Enabling/Disabling Datasets

By default, synthetic models will be created for all datasets in the Bundle when run_compute is called.  But you can disable a dataset to have run_compute ignore that dataset.  This is handled by a BoolParameter with the qualifier 'enabled' - and has a copy that lives in each set of compute options

Let's say we wanted to compute the orbit but not light curve - so we want to see enabled@lc01:

In [7]:
print b['enabled@lc01']

ParameterSet: 2 parameters
    enabled@lc01@preview@compute: True
   enabled@lc01@detailed@compute: True


as you can see, there is a copy for both of our compute options ('preview' and 'detailed').

If we know which set of compute options we'll be using, or only want to enable/disable for a given set, then we can do that:

In [8]:
b['enabled@lc01@preview'] = False
print b['enabled@lc01']

ParameterSet: 2 parameters
    enabled@lc01@preview@compute: False
   enabled@lc01@detailed@compute: True


or to enable/disable a dataset for all sets of compute options, we can use the set_value_all method:

In [9]:
b.set_value_all('enabled@lc01', False)
print b['enabled@lc01']

ParameterSet: 2 parameters
    enabled@lc01@preview@compute: False
   enabled@lc01@detailed@compute: False


If the enabled parameter is missing for a set of compute options - it is likely that that particular backend does not support that dataset type.

Running Compute
-----------------------

### Simplest Case

run_compute takes arguments for the compute tag as well as the model tag for the resulting synthetic model(s).  

You do not need to provide the compute tag if only 0 or 1 set of compute options exist in the Bundle.  If there are no compute options, the default PHOEBE 2.0 options will be added on your behalf and used.  If there is a single set of compute options, those will be assumed.  In our case, we have two compute options in the Bundle (with tags 'preview' and 'detailed') so we *must* provide an argument for compute.

If you do not provide a tag for the model, one will be created for you called 'latest'.  Note that any existing model with the same tag will immediately be overwritten once you call run_compute, so if you want to maintain the results from previous calls to run_compute, you must provide a NEW model tag.

In [10]:
b.run_compute(compute='preview')

<ParameterSet: 14 parameters | components: primary, secondary>

In [11]:
b.models

['latest']

### Storing Models

Now let's compute models for three different 'versions' of parameters.  By providing a model tag, we can keep the synthetics for each of these different runs in the bundle - which will be handy later on for plotting and comparing models.

In [12]:
b.set_value('incl@orbit', 90)
b.run_compute(compute='preview', model='run_with_incl_90') 

<ParameterSet: 14 parameters | components: primary, secondary>

In [13]:
b.set_value('incl@orbit', 85)
b.run_compute(compute='preview', model='run_with_incl_85')

<ParameterSet: 14 parameters | components: primary, secondary>

In [14]:
b.set_value('incl@orbit', 80)
b.run_compute(compute='preview', model='run_with_incl_80')

<ParameterSet: 14 parameters | components: primary, secondary>

We will now have three new sets of synthetics which can be compared, plotted, or removed.

In [15]:
b.models

['run_with_incl_85', 'latest', 'run_with_incl_80', 'run_with_incl_90']

### Running Compute with Multiple Sets of Options

So far we've seen how setting up different sets of compute options can be handy - 'preview' vs 'detailed', for example.  But there could also be situations where you want to use different sets of options per dataset.  Perhaps you have a high-precision follow-up light curve of an eclipse along with a lower-precision light curve over a longer time baseline.  So here you'd want to run 'detailed' on the high-precision light curve, but 'preview' on the lower-precision light curve.

You could of course call run_compute twice and create two separate models - but that isn't always convenient and will be a problem in the future when we want to fit data.

Instead we can send a list of compute options to run_compute.

A given dataset can only be enabled in up to 1 of the compute options we're sending to run_compute.  So let's take care of that first (if we don't, we'd get an error when trying to call run_compute):

In [16]:
print b['enabled@orb01']

ParameterSet: 2 parameters
   enabled@orb01@preview@compute: True
  enabled@orb01@detailed@compute: True


In [17]:
b.set_value_all('enabled@orb01@detailed', False)
b.set_value_all('enabled@orb01@preview', True)
print b['enabled@orb01']

ParameterSet: 2 parameters
   enabled@orb01@preview@compute: True
  enabled@orb01@detailed@compute: False


We probably have the same problem with 'lc01', but just didn't get far enough to raise the error.  So let's fix that as well

In [18]:
print b['enabled@lc01']

ParameterSet: 2 parameters
    enabled@lc01@preview@compute: False
   enabled@lc01@detailed@compute: False


In [19]:
b.set_value_all('enabled@lc01@detailed', True)
b.set_value_all('enabled@lc01@preview', False)
print b['enabled@lc01']

ParameterSet: 2 parameters
    enabled@lc01@preview@compute: False
   enabled@lc01@detailed@compute: True


So in this case, 'lc01' will be computed using the options in 'detailed' while 'orb01' will use the options in 'preview'.

In [20]:
b.run_compute(compute=['detailed', 'preview'], model='multiplecompute')

<ParameterSet: 16 parameters | kinds: ORB, LC>

In [21]:
b.models

['multiplecompute',
 'run_with_incl_85',
 'latest',
 'run_with_incl_80',
 'run_with_incl_90']

Accessing Synthetics from Models
----------------------------------------

The synthetics can be accessed by their dataset and model tags.

In [22]:
b['run_with_incl_90']

<ParameterSet: 14 parameters | components: primary, secondary>

In [23]:
b['primary@run_with_incl_90']

<ParameterSet: 7 parameters | qualifiers: time, y, x, vx, vy, vz, z>

In [24]:
b['x@primary@run_with_incl_90']

<Parameter: x=[  2.44929360e-16  -2.90949457e+00   3.99323263e+00  -2.57115044e+00
  -4.64371657e-01   3.20849277e+00  -3.93923101e+00   2.19803591e+00
   9.22463483e-01  -3.46410162e+00] solRad | keys: description, value, default_unit, visible_if, copy_for>

or of course through method access:

In [25]:
print b.get_value(qualifier='x', dataset='orb01', component='primary', model='run_with_incl_90')[:10]

[  2.44929360e-16  -2.90949457e+00   3.99323263e+00  -2.57115044e+00
  -4.64371657e-01   3.20849277e+00  -3.93923101e+00   2.19803591e+00
   9.22463483e-01  -3.46410162e+00]


For more details about the resulting Parameters in the model context, see the tutorial on the relevant dataset types:

* [Orbits (ORB)](ORB)
* [Meshes (MESH)](MESH)
* [Light Curves/Fluxes (LC)](LC)
* [Radial Velocities (RV)](RV)
* [Eclipse Timing Variations (ETV)](ETV)